In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

# import the two classes containing the algorithms
import scaling
import inference

In [2]:
# read in data
data = np.loadtxt('youtube_raw.txt',delimiter = ',')
data = data.astype(int)

In [3]:
data

array([[18681, 19936],
       [18681, 20564],
       [14414, 18681],
       ...,
       [15025, 13605],
       [ 9108, 11630],
       [13605, 14648]])

In [4]:
# for memory reasons, may need to truncate data
matrix = data[:30000,:]

# select largest strongly connected component
import networkx as nx
G = nx.DiGraph()

# pairwise comparison to build edges
rankings = list(map(tuple, matrix))
G.add_edges_from(rankings)

# largest strongly connected component
G = max(nx.strongly_connected_component_subgraphs(G), key=len)

# relabel and get updated rankings
G = nx.convert_node_labels_to_integers(G)
rankings = list(G.edges())

# return reduced Adjacency matrix
Adjacency = nx.adjacency_matrix(G).toarray()

In [5]:
Adjacency

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [6]:
# information from adjacency matrix
num_rankings = np.sum(Adjacency)
p = np.sum(Adjacency,1)/num_rankings
nb_items = Adjacency.shape[0]

In [7]:
# build the matrix used in matrix scaling algorithm
A = np.zeros((num_rankings,nb_items))

# build the matrix used in cvxpy
B = np.zeros((num_rankings,nb_items))
index = 0
for i in range(nb_items):
    length = np.sum(Adjacency[i,:])
    pair = np.where(Adjacency[i,:]==1)[0]
    A[index:index+length,i] = 1
    B[index:index+length,i] = 1
    A[np.arange(index,index+length),pair] = 1
    index += length

In [8]:
print('number of items is: ',nb_items)
print('number of comparisons is: ',num_rankings)

number of items is:  2156
number of comparisons is:  28134


In [9]:
rankings

[(0, 1),
 (0, 2),
 (0, 4),
 (0, 5),
 (0, 9),
 (0, 13),
 (0, 14),
 (0, 1940),
 (0, 137),
 (0, 1642),
 (0, 1875),
 (0, 363),
 (0, 1143),
 (0, 445),
 (0, 1333),
 (0, 392),
 (0, 1405),
 (0, 1981),
 (0, 376),
 (0, 1555),
 (1, 2087),
 (1, 715),
 (1, 1983),
 (1, 1997),
 (1, 202),
 (1, 1300),
 (1, 983),
 (1, 211),
 (1, 593),
 (1, 1487),
 (1, 2129),
 (1, 405),
 (1, 1805),
 (1, 1927),
 (2, 757),
 (2, 906),
 (2, 167),
 (3, 0),
 (3, 6),
 (3, 7),
 (3, 10),
 (3, 11),
 (3, 12),
 (3, 16),
 (3, 920),
 (3, 1550),
 (4, 212),
 (4, 1331),
 (4, 302),
 (4, 83),
 (4, 1224),
 (5, 149),
 (5, 89),
 (5, 1486),
 (5, 1712),
 (5, 1612),
 (6, 81),
 (6, 1446),
 (6, 789),
 (6, 1433),
 (6, 269),
 (6, 1320),
 (6, 1151),
 (6, 670),
 (6, 883),
 (6, 2032),
 (6, 2095),
 (6, 1818),
 (6, 675),
 (6, 527),
 (6, 1351),
 (7, 1014),
 (8, 867),
 (8, 1143),
 (9, 1889),
 (10, 1181),
 (10, 1110),
 (10, 151),
 (10, 706),
 (10, 347),
 (10, 779),
 (11, 1340),
 (12, 110),
 (13, 1712),
 (13, 154),
 (14, 1311),
 (14, 432),
 (15, 21),
 (15, 1

In [10]:
# MLE with scaling algorithm
start = time.time()
scaling_strength, scaled_A, iterations = scaling.iterative_scaling(A,p)
end = time.time()
print('Scaling algorithm converged in ',iterations, 'iterations and took ',end-start,'seconds.')

# MLE with iterative LSR algorithm
start = time.time()
ilsr_strength, iterations = inference.ilsr(nb_items,rankings)
end = time.time()
print('Iterative LSR algorithm converged in ',iterations, 'iterations and took ',end-start,'seconds.')

Scaling algorithm converged in  89 iterations and took  11.108970642089844 seconds.
Iterative LSR algorithm converged in  33 iterations and took  11.39851450920105 seconds.


In [11]:
np.sum(scaled_A,axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

In [12]:
scaling_strength/scaling_strength.sum()

array([0.00054287, 0.00061751, 0.00015826, ..., 0.00158299, 0.00052628,
       0.00088712])

In [13]:
ilsr_strength/ilsr_strength.sum()

array([0.00054287, 0.00061751, 0.00015826, ..., 0.00158299, 0.00052628,
       0.00088712])

In [14]:
max(abs(scaling_strength/scaling_strength.sum() - ilsr_strength/ilsr_strength.sum()))

9.255804000844747e-11

In [26]:
import cvxpy as cp

C = A-B

# Construct the problem.
x = cp.Variable((nb_items,))
#objective = cp.Maximize(cp.sum(-cp.log(1+cp.sum(cp.multiply(C,cp.exp(x)),axis=1))))
#objective = cp.Maximize(cp.sum(-cp.log(1+cp.matmul(C,cp.exp(x)))))
#objective = cp.Maximize(cp.sum(-cp.logistic(cp.log(C*cp.exp(x)))))
objective = cp.Minimize(cp.sum((C*cp.exp(x))))
#objective = cp.Minimize(cp.sum(C*x))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve(verbose=True)
# The optimal value for x is stored in `x.value`.
print(x.value)

c:\users\zhaon\appdata\local\programs\python\python36\lib\site-packages\cvxpy\problems\problem.py:781: RuntimeWarning: overflow encountered in long_scalars
  if self.max_big_small_squared < big*small**2:
c:\users\zhaon\appdata\local\programs\python\python36\lib\site-packages\cvxpy\problems\problem.py:782: RuntimeWarning: overflow encountered in long_scalars
  self.max_big_small_squared = big*small**2



ECOS 2.0.7 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -5.998e+02  +6e+03  7e-01  9e-01  1e+00  1e+00    ---    ---    0  0  - |  -  - 
 1  -2.945e+02  -8.918e+02  +5e+03  7e-01  9e-01  1e+00  8e-01  0.4010  6e-01   1  1  1 |  8  4
 2  -7.918e+02  -1.384e+03  +4e+03  6e-01  9e-01  1e+00  6e-01  0.9791  8e-01   1  1  1 | 11  0
 3  -1.414e+03  -1.999e+03  +3e+03  6e-01  9e-01  2e+00  5e-01  0.9791  8e-01   1  1  1 | 11  0
 4  -2.528e+03  -3.099e+03  +2e+03  5e-01  9e-01  2e+00  3e-01  0.5013  4e-01   1  1  1 |  6  3
 5  -4.880e+03  -5.417e+03  +1e+03  5e-01  8e-01  4e+00  2e-01  0.9791  6e-01   1  1  1 |  8  0
 6  -9.108e+03  -9.574e+03  +7e+02  4e-01  7e-01  5e+00  1e-01  0.6266  2e-01   1  1  1 |  4  2
 7  -9.600e+03  -1.002e+04  +5e+02  4e-01  6e-01  5e+00  8e-02  0.9791  8e-01   1  1  1 | 11  0
 8  -1.292e+04  -1.319e+04  +2e+02  2e-01  4e-

In [27]:
convex_strength = np.exp(x.value)
convex_strength = convex_strength/np.sum(convex_strength)

In [28]:
convex_strength

array([0.00022359, 0.00039536, 0.00050294, ..., 0.00339809, 0.00057785,
       0.00112052])